# Predictive analysis- Machine learning model ( ADASYN for Data Balancing + Ensemble Hard Voting)

# Importing Libraries

In [ ]:
#TODO
!pip install lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=90dd529f003da37de100b37462a4991f4ae60fcaf31e558edc4fd39f62aeb4bb
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


In [ ]:
# Bitcoin Heist Data Analysis with ADASYN, and LIME

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from scipy.stats import uniform, randint
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import ADASYN
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from lime import lime_tabular

# Exploratory Data Analysis

## Dataset loading

In [ ]:
df=pd.read_csv("BitcoinHeistData.csv")

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df["label"].value_counts()

,count
label,
white,2875284
paduaCryptoWall,12390
montrealCryptoLocker,9315
princetonCerber,9223
princetonLocky,6625
montrealCryptXXX,2419
montrealNoobCrypt,483
montrealDMALockerv3,354
montrealDMALocker,251


So only a small fraction of transactions are fraudulent. Our anomaly detection can work well.

In [ ]:
categorical_cols=df.select_dtypes(include="object").columns

In [ ]:
numerical_cols=df.select_dtypes(include="number").columns

In [ ]:
df.isnull().sum()

,0
address,0
year,0
day,0
length,0
weight,0
count,0
looped,0
neighbors,0
income,0
label,0


Woah! **No null values** :)

In [ ]:
df["address"].nunique()

2631095

In [ ]:
old_df=df.copy()

# Feature Engineering

In [ ]:
df.drop(columns=["address","year","day"],axis=1,inplace=True)

In [ ]:
for col in df.columns[:-1]:
    df[col]=df[col]

In [ ]:
X=df.drop(columns=["label"])
y=df["label"]

In [ ]:
X.columns

Index(['length', 'weight', 'count', 'looped', 'neighbors', 'income'], dtype='object')

In [ ]:
df.head()

,length,weight,count,looped,neighbors,income,label
0,18,0.008333,1,0,2,100050000.0,princetonCerber
1,44,0.000244,1,0,1,100000000.0,princetonLocky
2,0,1.000000,1,0,2,200000000.0,princetonCerber
3,72,0.003906,1,0,2,71200000.0,princetonCerber
4,144,0.072848,456,0,1,200000000.0,princetonLocky


## Number of Instances

In [ ]:
new_df=pd.DataFrame()
grouped=df.groupby("label")

In [ ]:
new_df["num_of_instances"]=grouped.size()

## Average

In [ ]:
df.columns

Index(['length', 'weight', 'count', 'looped', 'neighbors', 'income', 'label'], dtype='object')

In [ ]:
for col in X.columns:
    new_df[f"{col}_avg"]=grouped[col].mean()

In [ ]:
new_df.shape

(29, 7)

In [ ]:
new_df.head()

,num_of_instances,length_avg,weight_avg,count_avg,looped_avg,neighbors_avg,income_avg
label,,,,,,,
montrealAPT,11,67.636364,0.707728,2047.000000,734.090909,2.545455,3.719873e+08
montrealComradeCircle,1,144.000000,0.051214,1241.000000,0.000000,2.000000,2.033200e+08
montrealCryptConsole,7,43.428571,0.593306,831.714286,0.000000,2.000000,4.546334e+07
montrealCryptXXX,2419,47.447706,0.367505,791.848284,61.022323,2.011988,1.355343e+08
montrealCryptoLocker,9315,30.674396,0.888878,308.328824,100.981535,2.885346,1.840825e+09


## Standard Deviation

In [ ]:
for col in X.columns:
    new_df[f"{col}_std"]=grouped[col].agg(np.std).fillna(0)

<ipython-input-25-475c34e448b9>:2: FutureWarning: The provided callable <function std at 0x7e4b0b713eb0> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  new_df[f"{col}_std"]=grouped[col].agg(np.std).fillna(0)
<ipython-input-25-475c34e448b9>:2: FutureWarning: The provided callable <function std at 0x7e4b0b713eb0> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  new_df[f"{col}_std"]=grouped[col].agg(np.std).fillna(0)
<ipython-input-25-475c34e448b9>:2: FutureWarning: The provided callable <function std at 0x7e4b0b713eb0> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  new_df[f"{col}_std"]=grouped[col].agg(np.std).fillna(0)
<

## Minimum

In [ ]:
for col in X.columns:
    new_df[f"{col}_min"]=grouped[col].min()

## Maximum

In [ ]:
for col in X.columns:
    new_df[f"{col}_max"]=grouped[col].max()

In [ ]:
new_df=new_df.reset_index()
new_df.head()

,label,num_of_instances,length_avg,weight_avg,count_avg,looped_avg,neighbors_avg,income_avg,length_std,weight_std,...,count_min,looped_min,neighbors_min,income_min,length_max,weight_max,count_max,looped_max,neighbors_max,income_max
0,montrealAPT,11,67.636364,0.707728,2047.000000,734.090909,2.545455,3.719873e+08,73.145434,0.600596,...,1,0,1,57142857.0,144,1.666667,8076,8073,6,1.088599e+09
1,montrealComradeCircle,1,144.000000,0.051214,1241.000000,0.000000,2.000000,2.033200e+08,0.000000,0.000000,...,1241,0,2,203320001.0,144,0.051214,1241,0,2,2.033200e+08
2,montrealCryptConsole,7,43.428571,0.593306,831.714286,0.000000,2.000000,4.546334e+07,68.747987,0.410909,...,1,0,2,30000000.0,144,1.000000,3191,0,2,5.030000e+07
3,montrealCryptXXX,2419,47.447706,0.367505,791.848284,61.022323,2.011988,1.355343e+08,58.187904,0.434143,...,1,0,1,30000000.0,144,3.458951,9262,8489,15,1.080000e+09
4,montrealCryptoLocker,9315,30.674396,0.888878,308.328824,100.981535,2.885346,1.840825e+09,50.731602,1.555608,...,1,0,1,30000000.0,144,31.108593,6423,6418,79,4.450000e+11


# Data Preprocessing

In [ ]:
black_rows=df[df["label"]!="white"]
white_sample=df[df["label"]=="white"].sample(n=158587,random_state=42)
df=pd.concat([black_rows,white_sample]).sample(frac=1,random_state=50)
df.head()

## Label Encoding

We do label encoding of:
> * White label: 0
> * Ransomware: 1

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

df['label'] = (df['label'] != 'white').astype(int)


In [ ]:
df.head()

## Train Test Split

 Training data: 66%
> Test data: 33%

In [ ]:
X = df.drop('label', axis=1)
y = df['label']

## Data balancing (ADASYN-TL technique) And Splitting into Training and Testing dataset

In [ ]:
from sklearn.model_selection import train_test_split
adasyn = ADASYN(random_state=42)
X_balanced, y_balanced = adasyn.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.35, random_state=42, shuffle = True)

In [ ]:
y_balanced.value_counts()

## Standardization

We perform **normalization** on the train data and then scale test data accordingly.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
scaler.fit(X_train)

X_train_scaled=scaler.transform(X_train)
X_test_scaled=scaler.transform(X_test)

X_train_scaled=pd.DataFrame(X_train_scaled,columns=X_train.columns)
X_test_scaled=pd.DataFrame(X_test_scaled,columns=X_test.columns)

# Training

**This might take few minutes to run.**

In [ ]:
# Define models
models = {
    'lr': LogisticRegression(solver='saga', max_iter=1000),
    # 'svc': SVC(probability=True, random_state=42),
    'nb': GaussianNB(),
    # 'rf': RandomForestClassifier(random_state=42),
    'xgb': XGBClassifier(random_state=42)
}

## Hyperparameter tuning

In [ ]:
# Optimize XGBoost
param_grid_xgb = {
    'n_estimators': randint(50, 200),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.2),
    'subsample': uniform(0.7, 0.3),
    'colsample_bytree': uniform(0.7, 0.3)
}

In [ ]:
search_xgb = RandomizedSearchCV(XGBClassifier(random_state=42), param_distributions=param_grid_xgb,
                                n_iter=5, cv=3)
search_xgb.fit(X_train_scaled, y_train)
models['xgb'] = search_xgb.best_estimator_

In [ ]:
# Train voting classifier
voting_clf = VotingClassifier(
    estimators=[
        ('lr', models['lr']),
        ('nb', models['nb']),
        ('xgb', models['xgb'])
    ],
    voting='hard'
)
voting_clf.fit(X_train_scaled, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


VotingClassifier(estimators=[('lr',
                              LogisticRegression(max_iter=1000, solver='saga')),
                             ('nb', GaussianNB()),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.7493609897072195,
                                            device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None,...
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.10989214720120252,
                                            max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=9,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=91, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...))])

In [ ]:
# import numpy as np
# from scipy.stats import uniform, randint
# from sklearn.model_selection import RandomizedSearchCV
# from xgboost import XGBClassifier

# # Define the XGBoost classifier
# model = XGBClassifier()

# # Define the hyperparameter search space
# param_space = {
#     'n_estimators': randint(100, 1000),
#     'max_depth': randint(1, 10),
#     'learning_rate': uniform(0.01, 0.3),
#     'subsample': uniform(0.6, 0.4),
#     'colsample_bytree': uniform(0.6, 0.4)
# }

# # Perform random search
# random_search = RandomizedSearchCV(model, param_distributions=param_space, n_iter=10, cv=5)
# random_search.fit(X_train_scaled, y_train)

# # Print the best parameters and score
# print("Best parameters found: ", random_search.best_params_)
# print("Best score: ", random_search.best_score_)

# Evaluation

We use the following metrics for evaluation:
> * accuracy
> * precision
> * recall
> * f1 score

In [ ]:
# Evaluate voting classifier
results = {}
y_pred_voting = voting_clf.predict(X_test_scaled)
results['voting'] = {
    'accuracy': accuracy_score(y_test, y_pred_voting),
    'precision': precision_score(y_test, y_pred_voting),
    'recall': recall_score(y_test, y_pred_voting),
    'f1': f1_score(y_test, y_pred_voting)
}

In [ ]:
# Print results
for name, metrics in results.items():
    print(f"\nResults for {name.upper()}:")
    for metric, value in metrics.items():
        print(f"{metric.capitalize()}: {value:.2f}")